In [1]:
%matplotlib inline
import cartopy.feature
import cartopy.crs as ccrs
# import cartopy
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import numpy.ma as ma
import xarray as xr
import xesmf as xe
# import random
# from scipy import ndimage
import netCDF4 as ncdf
import hadcm3_deglac_glac1d.plotting as plotting
import hadcm3_deglac_glac1d.spreading as spreading
import hadcm3_deglac_glac1d.saving as saving
import hadcm3_deglac_glac1d.glac1d_toolbox as tb

## Test Spreading mask

In [2]:
ds_in = xr.open_dataset('/nfs/annie/eeymr/work/outputs/Proj_GLAC1D/dif_-26_0/teadv3.qrparm.GLAC1D_DEGLAC.nc', decode_times=False)
routed_mw = np.where(np.isnan(ds_in.discharge.values),0,  ds_in.discharge.values )
ds_wfix = xr.open_dataset("/nfs/annie/eeymr/work/data/Proj_GLAC1D/waterfix/teadv3.qrparam.waterfix.hadcm3.nc")
ds_lsm = xr.open_dataset("/nfs/annie/eeymr/work/data/Proj_GLAC1D/lsm/teadv3.qrparm.omask.nc")

spreaded_mw = spreading.spreading(routed_mw, ds_lsm, ds_wfix)

__ Spreading algorithm
____ Spreading in {'name': 'US_East_Coast', 'loc': 147 ocean cells in the region, 'region': 125 ocean cells in the region}
____ Spreading in {'name': 'Greenland_Arctic', 'loc': 163 ocean cells in the region, 'region': 163 ocean cells in the region}
____ Spreading in {'name': 'N_American_Arctic', 'loc': 390 ocean cells in the region, 'region': 355 ocean cells in the region}
____ Spreading in {'name': 'Gulf_of_Mexico', 'loc': 96 ocean cells in the region, 'region': 72 ocean cells in the region}
____ Spreading in {'name': 'East_Pacific', 'loc': 292 ocean cells in the region, 'region': 270 ocean cells in the region}
____ Spreading in {'name': 'Russia_Pacific', 'loc': 21 ocean cells in the region, 'region': 12 ocean cells in the region}
____ Spreading in {'name': 'LabradorSea_BaffinBay', 'loc': 228 ocean cells in the region, 'region': 175 ocean cells in the region}
____ Spreading in {'name': 'Atlantic_GreenlandIceland', 'loc': 116 ocean cells in the region, 'region': 

## Check final land sea mask

In [3]:
ds_lsm = xr.open_dataset('/nfs/annie/earpal/um/swsvalde/ancil/GLAC/21.0k/teadv3.qrparm.omask.nc')
lon_lsm, lat_lsm, lsm = ds_lsm.longitude.values, ds_lsm.latitude.values, ds_lsm.lsm.values

ds_final = xr.open_dataset('/nfs/annie/eeymr/work/outputs/Proj_GLAC1D/dif_-26_0_s/teadv3.qrparm.waterfix_GLAC1D_DEGLAC_s.nc', decode_times=False)
final_lsm = np.where(np.isnan(ds_final.discharge.values), 1,0)
                 
ds_previous = xr.open_dataset("/nfs/annie/eeymr/work/outputs/Proj_GLAC1D/Archive/differential_-2,28_100_sf/teadv3.qrparm.waterfix_GLAC1D_DEGLACs.nc", decode_times=False)
previous_lsm = np.where(np.isnan(ds_previous.discharge.values), 1,0)

ds_dump = xr.open_dataset("/nfs/annie/eeymr/work/data/Proj_GLAC1D/examples/dump_xosfb_teadv3.nc", decode_times=False)
dump_lsm = np.where(np.isnan(ds_dump.temp.isel(t=0).isel(depth=0).values[:,:-2]), 1,0)

FileNotFoundError: [Errno 2] No such file or directory: b'/nfs/annie/earpal/um/swsvalde/ancil/GLAC/21.0k/teadv3.qrparm.omask.nc'

In [ ]:
figMap,(axLSM, axDump) = plt.subplots(nrows=1, ncols=2, subplot_kw={'projection':projection_map}, figsize=(16,8))

axLSM.pcolormesh(lon_lsm, lat_lsm, lsm ,transform=ccrs.PlateCarree(), cmap="BrBG_r")
axLSM.set_title("teadv3 land sea mask")
axLSM.coastlines()
axLSM.set_global()

axDump.pcolormesh(lon_lsm, lat_lsm, dump_lsm ,transform=ccrs.PlateCarree(), cmap="BrBG_r")
axDump.set_title("dump land sea mask")
axDump.coastlines()
axDump.set_global()

In [ ]:
figMap, axDif = plt.subplots(nrows=1, ncols=1, subplot_kw={'projection':projection_map}, figsize=(16,8))

axDif.pcolormesh(lon_lsm, lat_lsm, dump_lsm - lsm,transform=ccrs.PlateCarree(), cmap="BrBG_r")
axDif.set_title("dump - lsm")
axDif.coastlines()
axDif.set_global()

In [ ]:
ds_lsm_fit = xr.open_dataset('/nfs/annie/eeymr/work/data/Proj_GLAC1D/lsm/teaea3.qrparm.omask.nc')
lsm_fit = ds_lsm_fit.lsm.values


figMap, axDif = plt.subplots(nrows=1, ncols=1, subplot_kw={'projection':projection_map}, figsize=(16,8))

axDif.pcolormesh(lon_lsm, lat_lsm, dump_lsm - lsm_fit,transform=ccrs.PlateCarree(), cmap="BrBG_r")
axDif.coastlines()
axDif.set_global()

In [ ]:
projection_map = ccrs.PlateCarree()

figMap,(axFinal,axPrevious, axDump) = plt.subplots(nrows=1, ncols=3, subplot_kw={'projection':projection_map}, figsize=(16,8))

cm = axFinal.pcolormesh(lon_lsm, lat_lsm, final_lsm[0] - lsm, transform=ccrs.PlateCarree())
axFinal.coastlines()
axFinal.set_global()

cm = axPrevious.pcolormesh(lon_lsm, lat_lsm, previous_lsm[100] - lsm ,transform=ccrs.PlateCarree())
axPrevious.coastlines()
axPrevious.set_global()

cm = axDump.pcolormesh(lon_lsm, lat_lsm, dump_lsm - lsm ,transform=ccrs.PlateCarree())
axDump.coastlines()
axDump.set_global()